# Coleta de dados
Nesta seção iremos coletar os dados por meio de web scraping e apis de outros sites

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
prefix = 'https://www.transfermarkt.com'
def getSoup(url):
    res = requests.get(url, headers = headers)
    return BeautifulSoup(res.content, 'html.parser')


### Função para realizar o web scraping das tableas de transferências no transfermarkt

In [2]:
def getClubUrl(url):
    url = url.replace('/transfers', '')  # Remove a substring '/transfers'
    match = re.search(r'/verein/(\d+)', url)
    if(not match):
        return "invalid";
    verein_id = match.group(1)  # Obtém o número após "/verein/"
    new_url = match.string[:match.start()] + \
        '/startseite' '/verein/' + verein_id
    return new_url


def eurValue(string):
    # Expressão regular para verificar se começa com '€' e termina com 'm' ou 'k'
    padrao = r'^€.*[mk]$'
    if re.match(padrao, string):
        return True
    else:
        return False


def eurToInt(string):
    valor_str = re.sub(r'[€mk]', '', string)  # Remove '€', 'm' e 'k' da string
    # Substitui a vírgula pelo ponto como separador decimal e converte para float
    valor_float = float(valor_str.replace(',', '.'))
    if 'm' in string:
        valor_int = int(valor_float * 1000000)  # Converte para milhões
    elif 'k' in string:
        valor_int = int(valor_float * 1000)  # Converte para milhares
    else:
        valor_int = int(valor_float)
    return valor_int


def getTransfersFromUrl(url, year):
    transferTable = []
    page = getSoup(url.format(year))
    tables = page.find_all('div', {'class': 'box'})
    tables = tables[3:]
    for table in tables:
        currTeamUrl = getClubUrl(table.find('a').get('href'))
        if(currTeamUrl == "invalid"):
            break;
        currTeam = table.find('a').get('title')
        arrivalsAndDepartures = table.find_all('table')

        arrivals = arrivalsAndDepartures[0].find_all('tr', recursive=True)
        arrivals = arrivals[1:]
        for transfer in arrivals:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text
            playerCountry = transfer[2].find('img').get('alt')
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubLeftUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubJoinedUrl = currTeamUrl
            clubLeft = transfer[6].find('a').get('title')
            clubJoined = currTeam
            teamTransferType = "out"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

        departures = arrivalsAndDepartures[1].find_all(
            'tr', recursive=True)
        departures = departures[1:]
        for transfer in departures:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text
            playerCountry = transfer[2].find('img').get('alt')
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubJoinedUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubLeftUrl = currTeamUrl
            clubLeft = currTeam
            clubJoined = transfer[6].find('a').get('title')
            teamTransferType = "in"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

    columns = ['playerName', 'playerLink', 'playerAge', 'playerPos', 'playerCountry',
               'playerMarketValue', 'transferFee', 'clubLeft', 'clubLeftUrl', 'clubJoined', 'clubJoinedUrl', 'teamTransferType']
    final_df = pd.DataFrame(transferTable, columns=columns)
    final_df['season'] = year
    final_df['playerAge'] = final_df['playerAge'].apply(
        lambda x: re.sub(r'\D+', '', str(x)))
    final_df['playerAge'] = final_df['playerAge'].astype(int)
    final_df['valuation'] = final_df['playerMarketValue'] - final_df['transferFee']
    return final_df


### Função para fazer o web scraping das tabelas de transferências de um determinado período e concatenar elas

In [3]:
def getTransfersInRange(begin, end, url):
    curr_df = getTransfersFromUrl(url, begin)
    for i in range(begin+1, end+1):
        curr_df = pd.concat(
            [curr_df, getTransfersFromUrl(url, i)]).reset_index(drop=True)
    return curr_df

### Obtendo dados das transferências para um range de temporadas da Serie A(demora 3 a 6 segundos por temporada, depende da internet)

In [4]:
sA = r'https://www.transfermarkt.com/serie-a/transfers/wettbewerb/IT1/plus/?saison_id={}&s_w=&leihe=0&intern=0'
sA_transfers = getTransfersInRange(2000, 2023, sA)
sA_transfers

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,clubJoinedUrl,teamTransferType,season,valuation
0,Gabriel Batistuta,/gabriel-batistuta/profil/spieler/5959,31,CF,Argentina,0,36150000,AC Fiorentina,/ac-florenz/startseite/verein/430,AS Roma,/as-rom/startseite/verein/12,out,2000,-36150000
1,Walter Samuel,/walter-samuel/profil/spieler/5938,22,CB,Argentina,0,20800000,CA Boca Juniors,/club-atletico-boca-juniors/startseite/verein/189,AS Roma,/as-rom/startseite/verein/12,out,2000,-20800000
2,Jonathan Zebina,/jonathan-zebina/profil/spieler/5942,21,CB,France,0,18400000,Cagliari Calcio,/cagliari-calcio/startseite/verein/1390,AS Roma,/as-rom/startseite/verein/12,out,2000,-18400000
3,Émerson,/emerson/profil/spieler/70,24,DM,Brazil,0,18000000,Bayer 04 Leverkusen,/bayer-04-leverkusen/startseite/verein/15,AS Roma,/as-rom/startseite/verein/12,out,2000,-18000000
4,Gianni Guigou,/gianni-guigou/profil/spieler/5954,25,DM,Uruguay,0,4000000,Club Nacional,/club-nacional/startseite/verein/866,AS Roma,/as-rom/startseite/verein/12,out,2000,-4000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774,Alessandro Di Pardo,/alessandro-di-pardo/profil/spieler/457674,23,RB,Italy,600000,1100000,Juventus Next Gen,/juventus-turin-u23/startseite/verein/41101,Cagliari Calcio,/cagliari-calcio/startseite/verein/1390,out,2023,-500000
4775,Christian Travaglini,/christian-travaglini/profil/spieler/685506,23,LB,Italy,200000,100000,Olbia Calcio 1905,/olbia-calcio-1905/startseite/verein/9826,Cagliari Calcio,/cagliari-calcio/startseite/verein/1390,out,2023,100000
4776,Raoul Bellanova,/raoul-bellanova/profil/spieler/357992,23,RB,Italy,5000000,7000000,Cagliari Calcio,/cagliari-calcio/startseite/verein/1390,Torino FC,/fc-turin/startseite/verein/416,in,2023,-2000000
4777,Alessio Cragno,/alessio-cragno/profil/spieler/12907,29,GK,Italy,3000000,3600000,Cagliari Calcio,/cagliari-calcio/startseite/verein/1390,AC Monza,/ac-monza/startseite/verein/2919,in,2023,-600000


In [6]:
sA_transfers = sA_transfers[sA_transfers['playerMarketValue'] != 0]
sA_transfers = sA_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
sA_transfers.to_csv('data/sA_transfers.csv', index=False)

OSError: ignored